In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing import image

In [2]:
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.20.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# eyepac-light-v2-512-jpg
img_size = (224, 224)
batch_size = 16

train_ds = tf.keras.utils.image_dataset_from_directory(
    "eyepac-light-v2-512-jpg/train",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="int"  # karena 0 dan 1
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "eyepac-light-v2-512-jpg/validation",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="int"
)

Found 8000 files belonging to 2 classes.


I0000 00:00:1772203922.126997  123786 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1399 MB memory:  -> device: 0, name: NVIDIA GeForce MX450, pci bus id: 0000:01:00.0, compute capability: 7.5


Found 770 files belonging to 2 classes.


In [4]:
print("Class names:", train_ds.class_names)

Class names: ['NRG', 'RG']


In [5]:
normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [6]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False  # freeze dulu

model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid")  # binary classification
])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 23s 2us/step


In [7]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [8]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10


2026-02-27 21:56:58.158486: I external/local_xla/xla/service/service.cc:163] XLA service 0x74548804f080 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2026-02-27 21:56:58.158531: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce MX450, Compute Capability 7.5
2026-02-27 21:56:58.315725: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-02-27 21:56:58.998833: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91900
2026-02-27 21:57:05.516824: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-27 21:57:05.660942: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: mea

  3/500 ━━━━━━━━━━━━━━━━━━━━ 26s 52ms/step - accuracy: 0.6076 - loss: 0.7819 

I0000 00:00:1772204227.413592  133258 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.6740 - loss: 0.6135

2026-02-27 21:57:43.298059: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-27 21:57:43.437310: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


500/500 ━━━━━━━━━━━━━━━━━━━━ 51s 78ms/step - accuracy: 0.7091 - loss: 0.5625 - val_accuracy: 0.7571 - val_loss: 0.5057
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 29s 58ms/step - accuracy: 0.7601 - loss: 0.4972 - val_accuracy: 0.7922 - val_loss: 0.4699
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 29s 58ms/step - accuracy: 0.7815 - loss: 0.4685 - val_accuracy: 0.8143 - val_loss: 0.4487
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 29s 58ms/step - accuracy: 0.7901 - loss: 0.4550 - val_accuracy: 0.7909 - val_loss: 0.4593
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 29s 58ms/step - accuracy: 0.7897 - loss: 0.4444 - val_accuracy: 0.8052 - val_loss: 0.4483
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 29s 58ms/step - accuracy: 0.8044 - loss: 0.4235 - val_accuracy: 0.7948 - val_loss: 0.4487
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 29s 58ms/step - accuracy: 0.8106 - loss: 0.4122 - val_accuracy: 0.7974 - val_loss: 0.4468
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 29s 58ms/step - accuracy: 0.8199 - loss: 0.4025 - val_accurac

In [9]:
base_model.trainable = True

# Bekukan sebagian awal layer, buka 30 layer terakhir
for layer in base_model.layers[:-30]:
    layer.trainable = False

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [11]:
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 55s 82ms/step - accuracy: 0.7283 - loss: 0.5860 - val_accuracy: 0.8104 - val_loss: 0.4928
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 35s 70ms/step - accuracy: 0.7905 - loss: 0.4478 - val_accuracy: 0.7987 - val_loss: 0.5102
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 35s 70ms/step - accuracy: 0.8204 - loss: 0.3891 - val_accuracy: 0.8091 - val_loss: 0.4725
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 35s 71ms/step - accuracy: 0.8455 - loss: 0.3468 - val_accuracy: 0.8078 - val_loss: 0.4505
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 35s 71ms/step - accuracy: 0.8716 - loss: 0.2939 - val_accuracy: 0.8078 - val_loss: 0.4347


In [15]:
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    preds = (preds > 0.5).astype(int).flatten()

    y_true.extend(labels.numpy())
    y_pred.extend(preds)

from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━

2026-02-27 22:52:02.792100: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [16]:
print("Train batches:", len(train_ds))
print("Validation batches:", len(val_ds))

# Hitung total gambar train
train_count = 0
for x, y in train_ds:
    train_count += x.shape[0]

val_count = 0
for x, y in val_ds:
    val_count += x.shape[0]

print("Total train images:", train_count)
print("Total validation images:", val_count)

Train batches: 500
Validation batches: 49
Total train images: 8000
Total validation images: 770


2026-02-27 22:57:06.355386: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    "eyepac-light-v2-512-jpg/test",
    image_size=(224, 224),
    batch_size=16,
    label_mode="int",
    shuffle=False  # PENTING: jangan shuffle
)

# normalisasi sama seperti train
test_ds = test_ds.map(lambda x, y: (x/255.0, y))

Found 770 files belonging to 2 classes.


In [18]:
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test Accuracy:", test_accuracy)

49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.8143 - loss: 0.4517
Test Accuracy: 0.8142856955528259


In [20]:
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    preds = (preds > 0.5).astype(int).flatten()

    y_true.extend(labels.numpy())
    y_pred.extend(preds)

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━

In [ ]:
# EyePACS-TRAIN-RG-3269
# eyepac-light-v2-512-jpg

img = image.load_img("eyepac-light-v2-512-jpg/test/RG/EyePACS-TRAIN-RG-3269.jpg", target_size=(224,224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  

prediction = model.predict(img_array)

print("Probabilitas glaucoma:", prediction[0][0])

if prediction[0][0] > 0.5:
    print("Prediksi: Glaucoma (1)")
else:
    print("Prediksi: Tidak Glaucoma (0)")

2026-02-27 23:45:06.208240: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-27 23:45:06.337926: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Probabilitas glaucoma: 0.914681
Prediksi: Glaucoma (1)


In [24]:
model.save("glaucoma_model.keras")
model.save("glaucoma_model.h5")

In [34]:
# EyePACS-TRAIN-RG-3269
# eyepac-light-v2-512-jpg

img = image.load_img("00022_g.png", target_size=(224,224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  

prediction = model.predict(img_array)

print("Probabilitas glaucoma:", prediction[0][0])

if prediction[0][0] > 0.5:
    print("Prediksi: Glaucoma (1)")
else:
    print("Prediksi: Tidak Glaucoma (0)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Probabilitas glaucoma: 0.8508127
Prediksi: Glaucoma (1)
